Dans ce TP, on va voir les bases de la manipulation des graphes.

Pour simplifier, on ne va considérer dans un premier temps que des graphes $G = ([|0;n-1|], A)$. 

# Matrice d'adjacence

Ainsi, on pourra considérer la matrice d'adjacence `mat : bool array array` telle que `mat.(i).(j)` soit égal à `true` si et seulement si $(i,j) \in A$.

On utiliser `make_matrix` puis remplir les arêtes une à une :

In [14]:
type mat_adj = bool array array

type mat_adj = bool array array

In [15]:
let mat_vide n : mat_adj =
    (* Renvoie la matrice d'adjacence du graphe vide sans arêtes *)
    Array.init n (fun t-> (Array.make n false))

val mat_vide : int -> mat_adj = <fun>

In [16]:
let mat_ajout_arete (m:mat_adj) x y : unit =
    (* Ajoute une arete x -> y *)
    m.(x).(y) <- true

val mat_ajout_arete : mat_adj -> int -> int -> unit = <fun>

In [17]:
let m = mat_vide 3 in
mat_ajout_arete m 0 1;
mat_ajout_arete m 0 2;
mat_ajout_arete m 1 2;
assert (m = [|[|false; true; true|]; [|false; false; true|]; [|false; false; false|]|]);
"OK"

- : string = "OK"

Comme ce n'est pas forcément très pratique de définir des graphes ainsi, on va commencer par réaliser une fonction `init_matrix` qui permettra d'appliquer une fonction `int -> int -> 'a` pour définir une matrice à la manière de `Array.init`.

In [18]:
let init_matrix n p f =
    Array.init n (fun i -> Array.init p (f i) )

val init_matrix : int -> int -> (int -> int -> 'a) -> 'a array array = <fun>

In [19]:
assert(init_matrix 2 2 (fun i j -> i+j) = [|[|0; 1|]; [|1; 2|]|]);
assert(init_matrix 2 2 (fun i j -> i = j) = [|[|true; false|]; [|false; true|]|]);
assert(init_matrix 1 3 (fun i j -> j) = [|[|0; 1; 2|]|]);
assert(init_matrix 3 1 (fun i j -> i) = [|[|0|]; [|1|]; [|2|]|]);
"OK"

- : string = "OK"

La fonction suivante renvoie la matrice d'adjacence pour $A = \left\{ (0,1), (1,2), \dots, (n-2, n-1), (n-1, 0) \right\}$.

In [20]:
let mat_cycle n =
    init_matrix n n (fun x y -> y = (x+1) mod n )

val mat_cycle : int -> bool array array = <fun>

In [21]:
mat_cycle 4;;

- : bool array array =
[|[|false; true; false; false|]; [|false; false; true; false|];
  [|false; false; false; true|]; [|true; false; false; false|]|]

In [22]:
assert (mat_cycle 4 = [|[|false; true; false; false|]; [|false; false; true; false|]; [|false; false; false; true|]; [|true; false; false; false|]|]);
assert (mat_cycle 2 = [|[|false; true|]; [|true; false|]|]);
"OK"

- : string = "OK"

In [23]:
let mat_complet n =
    (* Renvoie la matrice d'adjancence du graphe orienté complet
    à n arêtes, c'est-a-dire celui ayant toutes les arêtes possibles *)
    init_matrix n n (!=)

val mat_complet : int -> bool array array = <fun>

In [24]:
assert (mat_complet 3 = [|[|false; true; true|]; [|true; false; true|]; [|true; true; false|]|]);
"OK"

- : string = "OK"

Pour les fonctions suivantes, on demande d'extraire les voisins entrants et sortants sous forme d'une liste. On pourra utiliser une fonction auxilliaire récursive ou une référence sur une liste pour simuler une pile.

In [25]:
let mat_ex = [|[|false; true; true; true; false|]; [|false; false; true; false; false|];
  [|true; false; false; false; true|]; [|false; false; false; false; true|];
  [|false; false; true; true; false|]|];;

val mat_ex : bool array array =
  [|[|false; true; true; true; false|]; [|false; false; true; false; false|];
    [|true; false; false; false; true|];
    [|false; false; false; false; true|];
    [|false; false; true; true; false|]|]

In [26]:
let mat_voisins_entrants m x =
    let n = Array.length m in 
    let rec aux acc y =
        if y >= n then acc 
        else aux (if m.(y).(x) then (y::acc) else acc) (y+1) 
    in aux [] 0

val mat_voisins_entrants : bool array array -> int -> int list = <fun>

In [27]:
let mat_voisins_sortants m x =
    let n = Array.length m in 
    let rec aux acc y =
        if y >= n then acc 
        else aux (if m.(x).(y) then (y::acc) else acc) (y+1) 
    in aux [] 0

val mat_voisins_sortants : bool array array -> int -> int list = <fun>

In [28]:
let mat_degre_entrant m x =
    List.length (mat_voisins_entrants m x) 

val mat_degre_entrant : bool array array -> int -> int = <fun>

In [29]:
let mat_degre_sortant m x =
    List.length (mat_voisins_sortants m x) 

val mat_degre_sortant : bool array array -> int -> int = <fun>

In [30]:
let mat_ex = [|[|false; true; true; true; false|]; [|false; false; true; false; false|];
  [|true; false; false; false; true|]; [|false; false; false; false; true|];
  [|false; false; true; true; false|]|] in 
  List.map (mat_degre_entrant mat_ex) [0;1;2;3;4]

- : int list = [1; 1; 3; 2; 2]

In [31]:
let mat_ex = [|[|false; true; true; true; false|]; [|false; false; true; false; false|];
  [|true; false; false; false; true|]; [|false; false; false; false; true|];
  [|false; false; true; true; false|]|] in
assert (List.map (mat_degre_entrant mat_ex) [0;1;2;3;4] = [1; 1; 3; 2; 2]);
assert (List.map (mat_degre_sortant mat_ex) [0;1;2;3;4] = [3; 1; 2; 1; 2]);
"OK"

- : string = "OK"

# Graphes par matrice d'adjacence

Pour représenter un graphe, on va utiliser un tableau `'a array` pour les sommets et utiliser l'indice d'un élément pour accéder à la matrice d'adjacence.

Ainsi le graphe `A -> B <-> C` sera représenté par le tableau `[|"A";"B";"C"|]` et la matrice $\begin{pmatrix} 0 & 1 & 0 \\ 0 & 0 & 1 \\ 0 & 1 & 0 \end{pmatrix}$. Notons qu'on aurait pu permuter les éléments de $S$ et la matrice associée sera également permutée : `[|"C";"A";"B"|]` et $\begin{pmatrix} 0 & 0 & 1 \\ 0 & 0 & 1 \\ 1 & 0 & 0 \end{pmatrix}$.

On va devoir ainsi gérer a cohérence entre le tableau des sommets et la matrice d'adjacence le long de transformation.

On en déduit le type suivant. On va supposer **que chaque sommet est identifié uniquement par sa valeur**. Ainsi, dans l'exemple précédent `"A"` a un unique indice pour l'identifier. On remarque que le fait que les champs ne soient pas mutable signifie qu'il faudra créer un nouveau graphe pour changer l'ensemble de sommets.

In [19]:
type 'a graphem = { som : 'a array; mat : bool array array }

type 'a graphem = { som : 'a array; mat : bool array array; }

In [20]:
let graphem_vide () : 'a graphem = 
    (* renvoie un graphe sans sommet ni arếtes *)
    { som = [||];
    mat = [||];
    }

val graphem_vide : unit -> 'a graphem = <fun>

In [21]:
let graphem_isoles (s:'a array) : 'a graphem =
    (* Renvoie un graphe sans arêtes de sommets s *)
    {
        som = s;
        mat = Array.init (Array.length s) (fun _ -> Array.make (Array.length s) false);
    }

val graphem_isoles : 'a array -> 'a graphem = <fun>

In [22]:
let graphem_dim (g:'a graphem) : int * int =
    (* Renvoie le couple (n,p) où n est le nombre de sommets
    et p le nombre d'arêtes *)
    let asum f a =
        Array.fold_left (fun x y -> x + f y) 0 a in
    Array.length g.som, asum (asum (fun x -> if x then 1 else 0)) g.mat

val graphem_dim : 'a graphem -> int * int = <fun>

In [23]:
let g_ex = { som = [|'A';'B';'C';'D';'E'|]; mat = [|[|false; true; true; true; false|]; [|false; false; true; false; false|]; [|true; false; false; false; true|]; [|false; false; false; false; true|];
  [|false; false; true; true; false|]|] }

val g_ex : char graphem =
  {som = [|'A'; 'B'; 'C'; 'D'; 'E'|];
   mat =
    [|[|false; true; true; true; false|];
      [|false; false; true; false; false|];
      [|true; false; false; false; true|];
      [|false; false; false; false; true|];
      [|false; false; true; true; false|]|]}

In [24]:
assert (graphem_dim g_ex = (5, 9) );
"OK"

- : string = "OK"

La représentation n'étant pas très lisible, on aimerait pouvoir afficher les graphes sous la forme suivante :

```
Sommets: A B C D E 
Aretes: A -> B, A -> C, A -> D, B -> C, C -> A, C -> E, D -> E, E -> C, E -> D, 
```

On rappelle qu'avec `Jupyter` il faut un `printf` finissant par `\n%!` pour s'assurer de l'affichage.

In [25]:
let graphem_print g =
    (* Affiche un char graphem sous le format précédent *)
    Printf.printf "Sommets : ";
    Array.iter (Printf.printf "%c ") g.som;
    print_newline ();
    Printf.printf "Aretes :";
    Array.iteri (fun i t -> 
        Array.iteri (fun j x -> if x then  Printf.printf " %c -> %c," g.som.(i) g.som.(j) else ()) t ) g.mat
    ; print_newline()

val graphem_print : char graphem -> unit = <fun>

In [26]:
graphem_print (graphem_isoles [|'R';'A';'J';'E';'S';'H'|]);
graphem_print (g_ex);
graphem_isoles [|'R';'A';'J';'E';'S';'H'|], g_ex

Sommets : R A J E S H 
Aretes :
Sommets : A B C D E 
Aretes : A -> B, A -> C, A -> D, B -> C, C -> A, C -> E, D -> E, E -> C, E -> D,


- : char graphem * char graphem =
({som = [|'R'; 'A'; 'J'; 'E'; 'S'; 'H'|];
  mat =
   [|[|false; false; false; false; false; false|];
     [|false; false; false; false; false; false|];
     [|false; false; false; false; false; false|];
     [|false; false; false; false; false; false|];
     [|false; false; false; false; false; false|];
     [|false; false; false; false; false; false|]|]},
 {som = [|'A'; 'B'; 'C'; 'D'; 'E'|];
  mat =
   [|[|false; true; true; true; false|];
     [|false; false; true; false; false|];
     [|true; false; false; false; true|];
     [|false; false; false; false; true|];
     [|false; false; true; true; false|]|]})

A partir de maintenant, on va travailler sur les sommets puis en déduire leurs indices, on supposera que toutes les fonctions sont appelées avec des sommets valides.

In [27]:
exception Trouve of int

let graphem_indice (g:'a graphem) (x:'a) : int =
    (* Renvoie l'indice de x dans le graphe *)
    try
        for i = 0 to Array.length (g.som) - 1 do
            if g.som.(i) = x
            then raise (Trouve i)
        done;
        raise Not_found
    with Trouve i -> i 

exception Trouve of int
val graphem_indice : 'a graphem -> 'a -> int = <fun>

In [28]:
assert (graphem_indice g_ex 'C' = 2);
"OK"

- : string = "OK"

In [29]:
let graphem_test_arete (g:'a graphem) (x:'a) (y:'a) : bool =
    (* Test si le graphe contient une arete x -> y *)
    g.mat.(graphem_indice g x).(graphem_indice g y)

val graphem_test_arete : 'a graphem -> 'a -> 'a -> bool = <fun>

In [30]:
let graphem_ajoute_arete (g:'a graphem) (x:'a) (y:'a) : unit =
    (* Ajoute une arete x -> y *)
    g.mat.(graphem_indice g x).(graphem_indice g y) <- true

val graphem_ajoute_arete : 'a graphem -> 'a -> 'a -> unit = <fun>

In [31]:
let graphem_suppr_arete (g:'a graphem) (x:'a) (y:'a) : unit =
    (* Ajoute une arete x -> y *)
    g.mat.(graphem_indice g y).(graphem_indice g x) <- false

val graphem_suppr_arete : 'a graphem -> 'a -> 'a -> unit = <fun>

In [32]:
let g = graphem_isoles [|'A';'B';'C'|] in
graphem_ajoute_arete g 'A' 'B';
graphem_ajoute_arete g 'A' 'C';
graphem_ajoute_arete g 'C' 'A';
graphem_ajoute_arete g 'B' 'C';
assert (g.mat = [|[|false; true; true|]; [|false; false; true|]; [|true; false; false|]|])

- : unit = ()

Pour les fonctions suivantes, on pourra utiliser toutes les fonctions disponibles dans le module Array : https://ocaml.org/api/Array.html . Notamment `Array.append`.

In [47]:
let graphem_ajoute_sommet (g:'a graphem) (x:'a) : 'a graphem =
    (* Ajoute un nouveau sommet au graphe a la fin du tableau,
       il faudra changer la matrice *)
    {
        som = Array.append g.som [|x|] ;
        mat = Array.append 
            (Array.map (fun x -> Array.append x [|false|]) g.mat)
            [|(Array.make (Array.length g.som + 1) false)|];
    }

val graphem_ajoute_sommet : 'a graphem -> 'a -> 'a graphem = <fun>

In [48]:
let g = graphem_ajoute_sommet g_ex 'F' 
and g' = {som = [|'A'; 'B'; 'C'; 'D'; 'E'; 'F'|];
 mat =
[|[|false; true; true; true; false; false|];
    [|false; false; true; false; false; false|];
    [|true; false; false; false; true; false|];
    [|false; false; false; false; true; false|];
    [|false; false; true; true; false; false|];
    [|false; false; false; false; false; false|]|] } in
g, g', g.som = g'.som, g.mat = g'.mat

- : char graphem * char graphem * bool * bool =
({som = [|'A'; 'B'; 'C'; 'D'; 'E'; 'F'|];
  mat =
   [|[|false; true; true; true; false; false|];
     [|false; false; true; false; false; false|];
     [|true; false; false; false; true; false|];
     [|false; false; false; false; true; false|];
     [|false; false; true; true; false; false|];
     [|false; false; false; false; false; false|]|]},
 {som = [|'A'; 'B'; 'C'; 'D'; 'E'; 'F'|];
  mat =
   [|[|false; true; true; true; false; false|];
     [|false; false; true; false; false; false|];
     [|true; false; false; false; true; false|];
     [|false; false; false; false; true; false|];
     [|false; false; true; true; false; false|];
     [|false; false; false; false; false; false|]|]},
 true, true)

In [49]:
let g = graphem_ajoute_sommet g_ex 'F' in
assert (g = {som = [|'A'; 'B'; 'C'; 'D'; 'E'; 'F'|];
 mat =
  [|[|false; true; true; true; false; false|];
    [|false; false; true; false; false; false|];
    [|true; false; false; false; true; false|];
    [|false; false; false; false; true; false|];
    [|false; false; true; true; false; false|];
    [|false; false; false; false; false; false|]|]} );
"OK"

- : string = "OK"

In [153]:
let graphem_suppr_sommet (g:'a graphem) (x:'a) : 'a graphem =
    (* Supprime le sommet x. Attention cela va surement casser les correspondances avec les indices. 
     Une technique pourra consister à partir d'un graphe isole et a rajouter les aretes valides
     depuis g *)
     let filter a ind = 
         let r = Array.make (Array.length a - 1) a.(ind) in
         begin
         let rec aux i j = 
             if j < Array.length r then  
             if i = ind then aux (i+1) j 
             else begin r.(j) <- a.(i); aux (i+1) (j+1) end
         in 
         aux 0 0;
         r
         end
     in
    let g' = graphem_isoles (filter g.som (graphem_indice g x)) in
        Array.iteri ( fun i x->
            Array.iteri ( fun j y->
                if graphem_test_arete g x y then g'.mat.(i).(j) <- true )
                g'.som)
            g'.som;
         g'


val graphem_suppr_sommet : 'a graphem -> 'a -> 'a graphem = <fun>

In [154]:
assert (graphem_suppr_sommet g_ex 'C' = {som = [|'A'; 'B'; 'D'; 'E'|];
 mat = [|[|false; true; true; false|]; [|false; false; false; false|];
    [|false; false; false; true|]; [|false; false; true; false|]|]} );
"OK"

- : string = "OK"

In [155]:
not true

- : bool = false

In [173]:
let graphem_sous_graphe (g:'a graphem) (s:'a array) : 'a graphem  = 
    (* Renvoie le sous graphe induit par s sous-ensemble de g.som
    on pourra commencer par calculer le complémentaire : 'a graphemde s dans g.som 
    puis supprimer tous ces sommets *)
    let comp = Array.make (Array.length g.som - (Array.length s)) g.som.(0) in
    
         let rec aux i j = if i < Array.length g.som then  
             if Array.exists (fun x -> x = g.som.(i)) s then aux (i+1) j 
             else begin comp.(j) <- g.som.(i); aux (i+1) (j+1) end
         in 
         aux 0 0;
   
    let r = ref g in 
    Array.iter (fun x -> r := graphem_suppr_sommet !r x) comp;
    !r

val graphem_sous_graphe : 'a graphem -> 'a array -> 'a graphem = <fun>

In [175]:
assert (graphem_sous_graphe g_ex [|'A';'D'|] = {som = [|'A'; 'D'|]; mat = [|[|false; true|]; [|false; false|]|]});
"OK"

- : string = "OK"

# Listes d'adjacence

On va maintenant représenter les arêtes par un `ladj : int list array` où `ladj.(i)` donne la liste des indices des **voisins sortants** du sommet d'indice $i$.

On va reprendre les fonctions précédentes.

In [1]:
type liste_adj = int list array

type liste_adj = int list array

In [6]:
let ladj_vide n =
    Array.make n []

val ladj_vide : int -> 'a list array = <fun>

In [7]:
let ladj_ajout_arete l x y =
    l.(x) <- y :: l.(x)

val ladj_ajout_arete : 'a list array -> int -> 'a -> unit = <fun>

In [78]:
let ladj_suppr_arete l x y =
    l.(x) <- List.filter ((<>) y )  l.(x)

val ladj_suppr_arete : 'a list array -> int -> 'a -> unit = <fun>

In [4]:
let l = ladj_vide 5 in ladj_ajout_arete l 2 5; l

- : int list array = [|[]; []; [5]; []; []|]

In [5]:
let ladj_cycle n =
    let l = ladj_vide n in
    for i=0 to n-1 do
        ladj_ajout_arete l i ((i+1) mod n)
    done;
    l

val ladj_cycle : int -> int list array = <fun>

In [215]:
ladj_cycle 5

- : int list array = [|[1]; [2]; [3]; [4]; [0]|]

In [216]:
assert (ladj_cycle 4 = [|[1]; [2]; [3]; [0]|]);
"OK"

- : string = "OK"

In [220]:
let ladj_complet n =
    let l = ladj_vide n in
    for i=0 to n-1 do
        for j=0 to n-1 do
            if i!=j then
                ladj_ajout_arete l i j  else ()
            done
    done;
    l

val ladj_complet : int -> int list array = <fun>

In [221]:
assert (ladj_complet 4 = [|[3; 2; 1]; [3; 2; 0]; [3; 1; 0]; [2; 1; 0]|]);
"OK"

- : string = "OK"

In [222]:
let ladj_voisins_sortants l x = l.(x)

val ladj_voisins_sortants : 'a array -> int -> 'a = <fun>

In [9]:
let ladj_voisins_entrants l x =
    (* C'est plus compliqué ! On pourra utiliser List.filter et List.mem *)
    let n = Array.length l in
    List.filter (fun i-> List.mem x l.(i)) (List.init n (fun i->i))

val ladj_voisins_entrants : 'a list array -> 'a -> int list = <fun>

In [10]:
let l = [| [1;2;3]; [2]; [0;4]; [4]; [2;3] |] in
let l2 = ladj_voisins_entrants l 4 in
assert (l2 = [2;3] || l2 = [3;2]);
"OK"

- : string = "OK"

In [11]:
let ladj_degre_sortant l x = List.length l.(x)

val ladj_degre_sortant : 'a list array -> int -> int = <fun>

In [12]:
let ladj_degre_entrant l x =
    (* Ecrivez votre solution ici à la place de la ligne suivante. *)
    List.length (ladj_voisins_entrants l x)

val ladj_degre_entrant : 'a list array -> 'a -> int = <fun>

In [13]:
let ladj_ex = [|[3; 2; 1]; [2]; [4; 0]; [4]; [3; 2]|] in
assert (List.map (ladj_degre_entrant ladj_ex) [0;1;2;3;4] = [1; 1; 3; 2; 2]);
assert (List.map (ladj_degre_sortant ladj_ex) [0;1;2;3;4] = [3; 1; 2; 1; 2]);
"OK"

- : string = "OK"

In [33]:
let mat_to_ladj m =
    (* Convertit une matrice d'adjacence en liste d'adjacence *)
   Array.map (fun i -> mat_voisins_sortants m i) (Array.init (Array.length m) (fun i->i))

val mat_to_ladj : bool array array -> int list array = <fun>

In [34]:
let mat_ex = [|[|false; true; true; true; false|]; [|false; false; true; false; false|];
  [|true; false; false; false; true|]; [|false; false; false; false; true|];
  [|false; false; true; true; false|]|] in
assert (mat_to_ladj mat_ex = [|[3; 2; 1]; [2]; [4; 0]; [4]; [3; 2]|]);
"OK"

- : string = "OK"

In [40]:
let ladj_to_mat l =
    (* Convertit une liste d'adjacence en matrice d'adjacence *)
    let m = mat_vide (Array.length l) in
    Array.iteri 
        (fun x v -> List.iter (mat_ajout_arete m x) v) l;
        m

val ladj_to_mat : int list array -> mat_adj = <fun>

In [41]:
let mat_ex = [|[|false; true; true; true; false|]; [|false; false; true; false; false|];
  [|true; false; false; false; true|]; [|false; false; false; false; true|];
  [|false; false; true; true; false|]|] in
assert (ladj_to_mat [|[3; 2; 1]; [2]; [4; 0]; [4]; [3; 2]|] = mat_ex);
"OK"

- : string = "OK"

# Graphe par liste d'adjacence

In [42]:
type 'a graphel = 
{
    soml : 'a array;
    ladj : int list array
}

type 'a graphel = { soml : 'a array; ladj : int list array; }

In [43]:
(* Le g_ex sera alors representé par : *)
let g_ex_l = 
{
    soml = [|'A';'B';'C';'D';'E'|];
    ladj = [| [1;2;3]; [2]; [0;4]; [4]; [2;3] |]
}

val g_ex_l : char graphel =
  {soml = [|'A'; 'B'; 'C'; 'D'; 'E'|];
   ladj = [|[1; 2; 3]; [2]; [0; 4]; [4]; [2; 3]|]}

On va maintenant reprendre **toutes** les fonctions précédentes. **Attention** sans faire de conversions brutales depuis et vers les matrices d'adjacences.

In [44]:
let graphel_vide () = 
    {
        soml=[||];
        ladj=[||]
    }

val graphel_vide : unit -> 'a graphel = <fun>

In [45]:
let graphel_isoles s =
    {soml = s;
    ladj  = ladj_vide (Array.length s)}

val graphel_isoles : 'a array -> 'a graphel = <fun>

In [55]:
let graphel_dim g =
let n = Array.length g.soml in
    n,List.fold_left (fun acc ele -> acc+(ladj_degre_sortant g.ladj ele)) 0 (List.init n (fun i->i) )

val graphel_dim : 'a graphel -> int * int = <fun>

In [54]:
assert (graphel_dim g_ex_l = (5, 9) );
"OK"

- : string = "OK"

In [59]:
let graphel_print g =
     let n, _ = graphel_dim g in
    Printf.printf "Sommets: ";
    Array.iter (Printf.printf "%c ") g.soml;
    Printf.printf "\nAretes: ";
    for i = 0 to n-1 do
        List.iter (fun j -> Printf.printf "%c -> %c, " g.soml.(i) g.soml.(j)) g.ladj.(i)
    done;
    Printf.printf "\n%!" 

val graphel_print : char graphel -> unit = <fun>

In [60]:
graphel_print g_ex_l

Sommets: A B C D E 
Aretes: A -> B, A -> C, A -> D, B -> C, C -> A, C -> E, D -> E, E -> C, E -> D, 


- : unit = ()

In [62]:
exception Trouve of int

let graphel_indice (g:'a graphel) (x:'a) : int =
    (* Renvoie l'indice de x dans le graphe *)
     try
        for i = 0 to Array.length (g.soml) - 1 do
            if g.soml.(i) = x
            then raise (Trouve i)
        done;
        raise Not_found
    with Trouve i -> i 

exception Trouve of int
val graphel_indice : 'a graphel -> 'a -> int = <fun>

In [63]:
assert (graphel_indice g_ex_l 'C' = 2);
"OK"

- : string = "OK"

In [64]:
let graphel_test_arete (g:'a graphel) (x:'a) (y:'a) : bool =
    (* Test si le graphe contient une arete x -> y *)
    List.mem y (g.ladj.(graphel_indice g x))

val graphel_test_arete : int graphel -> int -> int -> bool = <fun>

In [68]:
let graphel_ajoute_arete (g:'a graphel) (x:'a) (y:'a) : unit =
    (* Ajoute une arete x -> y . ATTENTION ici il faut vérifier qu'elle n'existe pas deja *)
    let x,y = graphel_indice g x,graphel_indice g y in
    if List.mem y g.ladj.(x) then () 
    else ladj_ajout_arete g.ladj x y

val graphel_ajoute_arete : 'a graphel -> 'a -> 'a -> unit = <fun>

In [83]:
let graphel_suppr_arete (g:'a graphel) (x:'a) (y:'a) : unit =
    (* Supprime une arete x -> y *)
    ladj_suppr_arete g.ladj (graphel_indice g x) (graphel_indice g y)

val graphel_suppr_arete : 'a graphel -> 'a -> 'a -> unit = <fun>

In [84]:
let g = graphel_isoles [|'A';'B';'C'|] in
graphel_ajoute_arete g 'A' 'B';
graphel_ajoute_arete g 'A' 'C';
graphel_ajoute_arete g 'C' 'A';
graphel_ajoute_arete g 'B' 'C';
assert(g =  {soml = [|'A'; 'B'; 'C'|]; ladj = [|[2; 1]; [2]; [0]|]});
graphel_suppr_arete g 'C' 'A';
assert(g = {soml = [|'A'; 'B'; 'C'|]; ladj = [|[2; 1]; [2]; []|]});
"OK"

- : string = "OK"

In [85]:
let graphel_ajoute_sommet (g:'a graphel) (x:'a) : 'a graphel =
    (* Ajoute un nouveau sommet au graphe a la fin du tableau *)
    { soml = Array.append g.soml [|x|];
        ladj = Array.append g.ladj [| [] |]}

val graphel_ajoute_sommet : 'a graphel -> 'a -> 'a graphel = <fun>

In [86]:
graphel_ajoute_sommet g_ex_l 'F' (* doit valoir {soml = [|'A'; 'B'; 'C'; 'D'; 'E'; 'F'|];
 ladj = [|[1; 2; 3]; [2]; [0; 4]; [4]; [2; 3]; []|]} *)

- : char graphel =
{soml = [|'A'; 'B'; 'C'; 'D'; 'E'; 'F'|];
 ladj = [|[1; 2; 3]; [2]; [0; 4]; [4]; [2; 3]; []|]}

In [91]:
let rec suppr_adj l i =
    (* supprime le sommet i de la liste d'adjacence en décalant
    à j-1 les indices j > i *)
    List.map (fun x-> if x > i then x - 1 else x) (List.filter ((<>) i) l)

val suppr_adj : int list -> int -> int list = <fun>

In [92]:
suppr_adj [1;2;3;4;5] 3

- : int list = [1; 2; 3; 4]

In [93]:
assert (suppr_adj [1;2;3;4;5] 3 = [1;2;3;4]);
assert (suppr_adj [1;3;5] 3 = [1;4]);
"OK"

- : string = "OK"

In [101]:
let graphel_suppr_sommet (g:'a graphel) (x:'a) : 'a graphel =
    (* Supprime le sommet x *)
    let cut a i =
        let n = Array.length a in
        Array.append (Array.sub a 0 i) (Array.sub a (i+1) (n-1-i))
    in
    let i = graphel_indice g x in
    {
        soml = cut g.soml i;
        ladj = Array.map (fun l -> suppr_adj l i) (cut g.ladj i)
    }

val graphel_suppr_sommet : 'a graphel -> 'a -> 'a graphel = <fun>

In [102]:
graphel_suppr_sommet g_ex_l 'C'
(* doit valoir :
{soml = [|'A'; 'B'; 'D'; 'E'|]; ladj = [|[1; 2]; []; [3]; [2]|]}
*)

- : char graphel =
{soml = [|'A'; 'B'; 'D'; 'E'|]; ladj = [|[1; 2]; []; [3]; [2]|]}

# Bonus : affichage de graphe

Pour dessiner des graphes, on cherche à attribuer à chaque sommet des coordonnées dans le plan de manière ce que les arêtes soient representées par des segments *harmonieusement*. Une manière de faire cela est de considérer que chaque sommet est une charge ponctuelle de même charge, de même masse et que les arêtes sont des ressorts de même constante de raideur. Ne cherchez pas, ça n'a aucune validité physique, c'est juste un modèle.

On va commencer par placer les sommets aléatoirement et à chaque étape on va calculer le déplacement pendant un temps $dt$ de chaque sommet avec le PFD. Pour que ça converge, on va appliquer un coefficient $< 1$ sur le déplacement pour s'assurer qu'ils soient de plus en plus petits.

Au bout d'un certain nombre d'itération, on va obtenir un placement agréable.

Ici, nul besoin que la simulation soit physiquement réaliste, elle est d'**inspiration** physique. On peut donc utiliser des lois de la forme $\frac{A}{r^2}$ pour la répulsion des particules et $B r$ pour l'attraction dûe aux ressorts en choisissant $A$ et $B$ de manière agréable pour notre problème.

**Remarque** vous pouvez tout à fait faire une simulation avec les vraies lois physique et en introduisant un frottement fluide pour la convergence. Ça peut être pédagogiquement intéressant pour réviser les formules, mais ça ne fera pas un meilleur affichage de graphes.